# Generate the mvregfus configuration file and run the fusion
In this notwebook we specify the parameters for mvregus, generate a configuration file containing those and start the fusion process.

## Specify the parameters
Here we specify the parameters needed for the fusion. 

### Filepaths
First, we need to specify where the (unfused) images are and where we would like to save the fusions together with all the intermediate files. Moreover, we can choose to generate a file that contains the order in which the images were processed. This can be helpful, since the fused files will have a non-decriptive name.
* `filepaths`: path to folder containing the .czi images or list of paths to .czi images; if path to folder, all .czi files in that folder are processed.
* `out_dir`: folder in which the output files are saved, also all the intermediate results are stored there
* `generate_file_order`: bool, if True, a 'file_order.txt' file is generated in the output folder, that documents the order in which the files were processed.

In [1]:
filepaths = '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/'
out_dir = '/tower/fusions/cell_identity/et37/'
generate_file_order = True

### Image handling parameters
Next, we need to specify some properties of the images that are needed for the fusion. Those include which channels to use for the fusion, which views belong together and how the image is processed.
* `channels`: list, channels that should be fused
* `reg_channel`: int, channel that is used as reference for the fusion
* `ref_view`: int, reference view for the final fusion
* `registration pairs`: None or list, this can hold list of pairwise view indices to perform registration on; if None, we need to specify the number of volumes and views per volume to generate this list
* `n_volumes`: int, number of image volumes that make up the full image; needs to be specified only if `registration_pairs = None`
* `n_views`: int, number of views per volume; needs to be specified only if `registration_pairs = None`
* `raw_input_binning`: list [x, y, z], we can bin the input image e.g. for test purposes; this affects the final resolution
* `mv_registration_bin_factors`: list [x, y, z], bin factors used for the registration; binning during the registration does not affect the final resolution and speeds up the fusion significantly
* `mv_final_spacing`: float, final isotropic voxel size in micrometers
* `background level`: float, background level to subtract

In [2]:
channels = [0, 1]
reg_channel = 1
ref_view = 0
registration_pairs = None
n_volumes = 4
n_views = 4
raw_input_binning = [1,1,1]
mv_registration_bin_factors = [4,4,1]
mv_final_spacing = 1
background_level = 200

### Fusion parameters
Finally, we specify the parameters of the fusion, such as the fusion method and what kind of weights should be used.
* `fusion_method`: 'weighted_average': weighted average of views using the given weights (fast), 'LR': Lucy-Richardson multi-view deconvolution (slow but better image quality)
* `fusion_weights`: 'blending': uniform weights with blending at the stack borders, 'dct': weights derived from DCT image quality metric (slower but better)

In [3]:
fusion_method = 'LR'
fusion_weights = 'dct'

## Generate the configuration file
Here we write the parameters that we specified above into `mvregfus_config.json` which will be read by `mvregfus_run.py`. The .json file will be located in the `out_dir` specified above.

In [4]:
import os
import json
parameter_dict = {'filepaths': filepaths,
                  'out_dir': out_dir,
                  'generate_file_order': True,                
                  'channels': channels,
                  'reg_channel': reg_channel,
                  'ref_view': ref_view,
                  'registration_pairs': registration_pairs,
                  'n_volumes': n_volumes,
                  'n_views': n_views,
                  'raw_input_binning': raw_input_binning,
                  'mv_registration_bin_factors': mv_registration_bin_factors,
                  'mv_final_spacing': [mv_final_spacing]*3,
                  'background_level': background_level,
                  'fusion_method': fusion_method,
                  'fusion_weights': fusion_weights,
                 }
with open(os.path.join(out_dir, 'mvregfus_config.json'), "w") as outfile:  
    json.dump(parameter_dict, outfile)


In [5]:
import glob
glob.glob(os.path.join(filepaths, '*.czi'))

['/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi',
 '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish02.czi',
 '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish03.czi',
 '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish04.czi']

## Run the fusion
Finally, we start the fusion by calling `mvregfus_run.py` with the parameters in the config file.

In [6]:
import mvregfus_run
parameters = mvregfus_run.read_config_file(os.path.join(out_dir, 'mvregfus_config.json'))
parameters

INFO: successfully registered image array for dask distributed serialization
http://localhost:43243
]0;multi-view fusion: http://localhost:43243

/data/shared/miniconda3/envs/mvregfus/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43243 instead
  http_address["port"], self.http_server.port


{'filepaths': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/',
 'out_dir': '/tower/fusions/cell_identity/et37/',
 'generate_file_order': True,
 'channels': [0, 1],
 'reg_channel': 1,
 'ref_view': 0,
 'registration_pairs': None,
 'n_volumes': 4,
 'n_views': 4,
 'raw_input_binning': [1, 1, 1],
 'mv_registration_bin_factors': [4, 4, 1],
 'mv_final_spacing': [1, 1, 1],
 'background_level': 200,
 'fusion_method': 'LR',
 'fusion_weights': 'dct'}

In [ ]:
mvregfus_run.run_fusion(parameters)

These pairs of keys will be registered:
[[0, 1],
 [1, 2],
 [3, 0],
 [2, 3],
 [0, 4],
 [4, 5],
 [5, 6],
 [7, 4],
 [6, 7],
 [4, 8],
 [8, 9],
 [9, 10],
 [11, 8],
 [10, 11],
 [8, 12],
 [12, 13],
 [13, 14],
 [15, 12],
 [14, 15]]
They refer to the keys in 'view_dict':
{0: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi',
     'ill': 2,
     'view': 0},
 1: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi',
     'ill': 2,
     'view': 1},
 2: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi',
     'ill': 2,
     'view': 2},
 3: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi',
     'ill': 2,
  

These pairs of keys will be registered:
[[0, 1],
 [1, 2],
 [3, 0],
 [2, 3],
 [0, 4],
 [4, 5],
 [5, 6],
 [7, 4],
 [6, 7],
 [4, 8],
 [8, 9],
 [9, 10],
 [11, 8],
 [10, 11],
 [8, 12],
 [12, 13],
 [13, 14],
 [15, 12],
 [14, 15]]
They refer to the keys in 'view_dict':
{0: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish04.czi',
     'ill': 2,
     'view': 0},
 1: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish04.czi',
     'ill': 2,
     'view': 1},
 2: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish04.czi',
     'ill': 2,
     'view': 2},
 3: {'filename': '/mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish04.czi',
     'ill': 2,
  

/data/shared/MVRegFus/mvregfus/io_utils.py:254: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  tmpFile = h5py.File(path)


DECORATOR local... readStackFromMultiviewMultiChannelCzi
reading /mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi view 4 ch 1 ill 2
fusing 2 illuminations using sample center
fusing illuminations planewise
[########################################] | 100% Completed | 13.2s
Binning down raw input by xyz factors [1, 1, 1]
old shape: 201 1920 1400
DECORATOR local... bin_stack
new shape: 201 1920 1400
DECORATOR local... bin_stack
DECORATOR local... register_linear_elastix
producing /tower/fusions/cell_identity/et37/mv_params_000_000_vfix000_vmov004.prealignment.h5
compute clahe with kernel size 10
compute clahe with kernel size 10
DECORATOR local... calc_stack_properties_from_views_and_params
stack size in FFT translation registration: [174, 88, 350]


/data/shared/MVRegFus/mvregfus/io_utils.py:254: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  tmpFile = h5py.File(path)


...

transforming and streaming to file: /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v000_c00.image.ims
Writing into /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v000_c00.image.ims
[########################################] | 100% Completed |  1min 18.5s
DECORATOR local... readStackFromMultiviewMultiChannelCzi
reading /mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi view 1 ch 0 ill 2
fusing 2 illuminations using sample center
fusing illuminations planewise
[########################################] | 100% Completed | 31.5s
Binning down raw input by xyz factors [1, 1, 1]
old shape: 496 1920 1400
DECORATOR local... bin_stack
new shape: 496 1920 1400
matrices [[ 3.76991119e-07  0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  3.76991119e-07]] [[ 3.76991119e-07  0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  1.81

[########################################] | 100% Completed |  1min 13.3s
DECORATOR local... readStackFromMultiviewMultiChannelCzi
reading /mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi view 8 ch 0 ill 2
fusing 2 illuminations using sample center
fusing illuminations planewise
[########################################] | 100% Completed | 17.0s
Binning down raw input by xyz factors [1, 1, 1]
old shape: 261 1920 1400
DECORATOR local... bin_stack
new shape: 261 1920 1400
matrices [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] [[1.         0.         0.        ]
 [0.         1.81450732 0.        ]
 [0.         0.         1.81450732]]
offsets [-4.26137857 -4.82127836  2.66128886] [ -346.24976041 -2474.           -32.3348968 ]
transforming and streaming to file: /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v008_c00.image.ims
Writing into /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v008_c00

fusing 2 illuminations using sample center
fusing illuminations planewise
[########################################] | 100% Completed | 20.2s
Binning down raw input by xyz factors [1, 1, 1]
old shape: 313 1920 1400
DECORATOR local... bin_stack
new shape: 313 1920 1400
matrices [[-1.25663707e-07  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00 -1.25663707e-07]] [[-1.25663707e-07  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.81450732e+00  0.00000000e+00]
 [-1.81450732e+00  0.00000000e+00 -2.28017716e-07]]
offsets [-223.04389168   -8.33978181 -736.30927562] [ -274.37994153 -3410.          1414.56602368]
transforming and streaming to file: /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v015_c00.image.ims
Writing into /tower/fusions/cell_identity/et37/mv_transf_view_000_000_v015_c00.image.ims
[########################################] | 100% Completed |  1min  9.4s
fusion block overlap:  16.0
calculating 

/data/shared/miniconda3/envs/mvregfus/lib/python3.7/site-packages/scipy/ndimage/filters.py:1528: RuntimeWarning: All-NaN slice encountered
  cval, origins, extra_arguments, extra_keywords)


adapting weights to 16 views
[########################################] | 100% Completed | 36.0s
CuPy available, using single host thread for fusion

fusing views...
Writing into /tower/fusions/cell_identity/et37/mv_000_000_c00.ims
[########################################] | 100% Completed |  1hr  0min 44.3s
DECORATOR local... readStackFromMultiviewMultiChannelCzi
reading /mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-et37-sytoxFR-48hpf-fish01.czi view 0 ch 1 ill 2
fusing 2 illuminations using sample center
fusing illuminations planewise
[########################################] | 100% Completed | 24.6s
Binning down raw input by xyz factors [1, 1, 1]
old shape: 383 1920 1400
DECORATOR local... bin_stack
new shape: 383 1920 1400
DECORATOR local... bin_stack
DECORATOR local... readStackFromMultiviewMultiChannelCzi
reading /mnt/zmb/data/m.brambach/zeiss_lightsheet_transfer_20201116_1716/2020-11-16-et37-sytoxFR-48hpf/2020-11-16-e